# 04 - Contested Catch Index

Compute contested catch index metric.

Alright… deep breath 😮‍💨
Time to build **the REAL Contested-Catch Index (CCI) playbook** — the one that actually fits *our project*, *our reconstruction*, and *our core objective*:

👉 **“Show the invisible battle between receiver and defender while the ball is in the air.”**

No lists.
No leaderboards.
No random charts.
Just **movement, interaction, and coaching insight.**

Let’s build it properly.

---

# 🎯 **CCI Playbook — The Interaction Metric at the Catch Moment**

This playbook is written so we **don’t drift**, **don’t guess**, and **don’t compute irrelevant noise**.
It’s built strictly around:

* our existing reconstructed dataset
* our earlier two metrics (Separation Gain + TTI)
* our coaching story chain: **Throw → Pursuit → Interaction → Impact**

---

# 🟥 **Metric 3 — Contested-Catch Index (CCI)**

### *“How tight was the fight at the moment the ball arrives?”*

CCI is your **impact-moment metric** — the last stop in the chain after SG (what the WR did) and TTI (what the defender did).

---

# 🧭 **PLAYBOOK OVERVIEW**

### CCI MUST capture 3 things at the catch moment:

1. **How close the defender is**
2. **How many defenders are close enough to challenge**
3. **Whether the defender is closing fast or fading away**

Together, these tell the coaching story:

👉 *“Did the WR face tight coverage?”*
👉 *“Were defenders in position?”*
👉 *“Was the catch easy or contested?”*

---

# 🧩 **STAGE 0 — Inputs Checklist (must exist)**

### Required columns from reconstructed frame-level data:

| Feature                                           | Why needed                   | Present?                         |
| ------------------------------------------------- | ---------------------------- | -------------------------------- |
| `game_id`, `play_id`, `nfl_id`                    | grouping                     | ✔️                               |
| `x`, `y`, `frame_id`                              | locations over time          | ✔️                               |
| `phase`                                           | pre/post throw window        | ✔️                               |
| `player_role` or position                         | identify WR vs defenders     | ✔️                               |
| `dist_to_land`                                    | used earlier (optional here) | ✔️                               |
| `radial_v` OR closing speed                       | defender movement direction  | ✔️                               |
| `separation_time_series` OR ability to compute it | need sep at catch            | ✔️(after adding missing columns) |

### Additional computed signals we MUST extract:

* **sep_at_catch_end** → we already have this
* **nearest defender ID at catch**
* **number of defenders within radius** (we’ll compute)
* **closing speed at catch frame** (we already can compute via dx/dy or radial_v)

We’re good.

---

# 🧠 **CCI Formula (adapted for our project)**

We’ll use a **scale → normalize → logistic** structure:

[
\text{CCI} = 1 - \sigma\left( w_1 \cdot \text{sep}_{catch}

* w_2 \cdot n_{defenders}
* w_3 \cdot v_{close} \right)
  ]

where:

* low separation → makes inside sigmoid more negative → **CCI ↑ (tight)**
* high number of defenders → tighter → **CCI ↑**
* faster closing speed → tighter → **CCI ↑**

Sigmoid ensures the output lies between 0–1.

---

# 🧭 **STAGE 1 — Identify the Catch-Moment Frame**

**Purpose:** pick the *final post-throw frame* for each player — that is the moment of impact.

### Action

```
catch_frame = df[df.phase=="post_throw"].groupby(['game_id','play_id']).frame_id.max()
```

This gives us `t_catch` per play.

### Validation

Plot a few plays to ensure the last output frame aligns with catch/end-of-flight.

---

# 🧭 **STAGE 2 — Compute Required Catch-Moment Features**

### For the targeted WR in each play:

1. **Separation at catch**
   Already stored in `sep_at_catch_end`.

2. **Nearest defender at catch**
   Compute Euclidean distance between WR and each defender at `t_catch`.

3. **Number of defenders within challenge radius**
   Use radius = **3 yards** (NFL standard for contest zone).

4. **Defender closing speed**
   Extract `v_close` or `radial_v` from the closing defender.

---

# 🧭 **STAGE 3 — Normalize Inputs**

Purpose: avoid certain features dominating the sigmoid.

Normalize to 0–1:

* sep_norm = sep / 10 (10 yards separation is enormous → treated as “very open”)
* n_def_norm = min(n_def, 3) / 3
* vclose_norm = (v_close + 8) / 16 → maps −8..8 yd/s to 0..1

---

# 🧭 **STAGE 4 — Compute CCI**

### Weighted logistic model:

[
z = w_1 \cdot sep_norm + w_2 \cdot n_def_norm + w_3 \cdot vclose_norm
]

Use simple equal weights:

```
w1 = 0.4
w2 = 0.3
w3 = 0.3
```

Then:

```
CCI = 1 - sigmoid(z)
```

---

# 🧭 **STAGE 5 — Categorize for Coaches**

| CCI       | Label        | Meaning            |
| --------- | ------------ | ------------------ |
| > 0.75    | **Tight**    | Defender contested |
| 0.40–0.75 | **Moderate** | Pressure present   |
| < 0.40    | **Open**     | WR free            |

---

# 🧭 **STAGE 6 — Visualization (COACH-FRIENDLY)**

CCI must show **movement + danger**, not numbers.

### View 1 — Catch-Moment Snapshot

* Show WR location
* Show nearest defenders
* Circle radius = 3y
* Color WR based on “tight/open”

### View 2 — Multi-Frame Convergence Animation

Combine:

* WR path
* Defender path
* Separation shrinking
* Closing speed arrows
* A final freeze frame with “CCI = 0.82 (Tight)” overlay

### View 3 — Film-Room Chalk Diagram

One static image:

* WR path (blue)
* DB path (red)
* Catch circle
* CCI label

This will be the diagram coaches love.

---

# 🟩 **How CCI Connects to Our Previous Metrics**

This is important — the story must flow.

### Before Throw → Separation Gain

“Did the WR create space?”

### Mid-Flight → TTI

“Did the defender react quickly and take a good pursuit path?”

### End-Flight → CCI

“How contested was the catch moment?”

So our full chain becomes:

> **SG → TTI → CCI**
> “space created → space chased → space contested”

This is a **complete dynamic picture** of a pass play.

---

# 🟦 **FINAL OUTPUT FILE STRUCTURE**

For each **(game_id, play_id, nfl_id for targeted WR)** store:

* sep_at_throw
* sep_at_catch
* sep_gain
* closing_speed_at_catch
* defenders_within_3yd
* nearest_defender_id
* CCI
* CCI_category
* pass_result

This becomes `processed/metric_cci.parquet`.

